# Regression lineaire multiple

## 1. Importation des données nettoyées et configuration du data set

In [188]:
# On importe les librairies dont on aura besoin pour ce tp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

pd.set_option('display.max_columns', None)

# On charge le dataset
df = pd.read_csv("../csv/clean.csv", sep=',')

In [189]:
df.shape

(1460, 76)

## 1. Choix du modèle de machine learning

on passe de la regression lineaire simple à la regression lineaire multiple

## 2. Préparation des données

### 2.a échantillonage

In [190]:
# Le jeu de données étant petit il n'est pas nécessaire de réaliser l'apprentissage sur un échantillon

#df.sample(n=10000)

### 2.b Sélection de la variable cible et des variables à explicatives

Il est apparu que la seule variable GRLiveArea ne suffit pas à calculer le SalePrice, je vais donc rajouter les autres variables quantitatives relevées dans l'analyse, c'est a dire celle avec le coefficient de corrélation le plus fort.

GrLivArea:  Above grade (ground) living area square feet, indice de corrélation de 0.71

OverallQual:Rates the overall material and finish of the house , indice de corrélation de 0.80

GarageCars: Size of garage in car capacity, indice de corrélation de 0.64

GarageArea:Size of garage in square feet, indice de corrélation de 0.62

On définit à présent notre variable cible

In [191]:
y = df.SalePrice

Ainsi que nos variables explicatives

In [192]:
X = df[["GrLivArea","OverallQual","GarageArea","GarageCars"]]

## 3. Division du jeu de données en training/validation/testing sets

In [193]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(X, y, train_size=0.8)

## 4. Apprentissage sur le training set

#### On implémente la regression linéaire

In [194]:
from sklearn import linear_model
regr = linear_model.LinearRegression()
regr.fit(xtrain, ytrain)

LinearRegression()

#### On obtient ainsi les parmètres β1 et β0 de notre modèle

In [195]:
print("Le coefficient β1 est égal à {} tandis que l'intercept β0 est égal à {}"
      .format(round(regr.coef_[0],2),round(regr.intercept_)))

Le coefficient β1 est égal à 46.38 tandis que l'intercept β0 est égal à -101693


#### On peut à présent réaliser des prédictions

In [196]:
i=45

print("selon nos données, un logement de {} pieds2, de qualité {}, avec un garage de {} m2, et avec {}garages a un prix de vente de {} €"
      .format(df.iloc[i].GrLivArea,df.iloc[i].OverallQual,df.iloc[i].GarageArea,df.iloc[i].GarageCars,df.iloc[i].SalePrice))

x_to_predict = (df.iloc[i].GrLivArea,df.iloc[i].OverallQual,df.iloc[i].GarageArea,df.iloc[i].GarageCars)
x_df =  pd.DataFrame({'SalePrice': [x_to_predict[0]],'GrLivArea': [x_to_predict[1]],'GarageArea': [x_to_predict[2]],'GarageCars': [x_to_predict[3]]})

print("Selon le modèle, un logement de {} pieds2, de qualité {}, avec un garage de {}m2, et avec {}garages doit avoir un prix de vente de {} €"
      .format(x_to_predict[0],x_to_predict[1],x_to_predict[2],x_to_predict[3], round(regr.predict(x_df)[0])))


selon nos données, un logement de 1752 pieds2, de qualité 9, avec un garage de 576 m2, et avec 2garages a un prix de vente de 319900 €
Selon le modèle, un logement de 1752 pieds2, de qualité 9, avec un garage de 576m2, et avec 2garages doit avoir un prix de vente de 281143 €


## 5. Evaluation du modèle sur le testing set

In [197]:
from sklearn.metrics import mean_squared_error, r2_score

#### évaluation à partir du R2

In [198]:
ytrainpred = regr.predict(xtrain)
ytestpred = regr.predict(xtest)

print ("Pour les données d'entrainement le R2 vaut {} alors que pour les données de test, il est de {}" 
      .format(round(r2_score(ytrain, ytrainpred),3), round(r2_score(ytest, ytestpred),3)))



Pour les données d'entrainement le R2 vaut 0.739 alors que pour les données de test, il est de 0.752


Le coefficient de détermination R2 est monté à plus de 0.735, c'est un résultat solide. 
les R2 des jeux d'entrainement et de test sont proche, le modèle est donc robuste.

#### évaluation à partir du RMSE

In [199]:
print ("Pour les données d'entrainement le RMSE vaut {} alors que pour les données de test, il est de {}" 
      .format(round(mean_squared_error(ytrain, ytrainpred, squared=False),3), 
              round(mean_squared_error(ytest, ytestpred, squared=False),3)))

Pour les données d'entrainement le RMSE vaut 40915.892 alors que pour les données de test, il est de 38254.292


A nouveau on remarque qu'il y a peu de perte entre les deux modèles

# 6. regression lineaire de lasso

In [200]:
from sklearn import linear_model
regr = linear_model.Lasso(alpha=5)
regr.fit(xtrain, ytrain)

print("Le coefficient β1 est égal à {} tandis que l'intercept β0 est égal à {}"
      .format(round(regr.coef_[0],2),round(regr.intercept_)))


Le coefficient β1 est égal à 46.39 tandis que l'intercept β0 est égal à -101678


## 6.1 Evaluation du modèle sur le testing set

In [201]:
ytrainpred = regr.predict(xtrain)
ytestpred = regr.predict(xtest)

print ("Pour les données d'entrainement le R2 vaut {} alors que pour les données de test, il est de {}" 
      .format(round(r2_score(ytrain, ytrainpred),3), round(r2_score(ytest, ytestpred),3)))


Pour les données d'entrainement le R2 vaut 0.739 alors que pour les données de test, il est de 0.752


In [202]:
print ("Pour les données d'entrainement le RMSE vaut {} alors que pour les données de test, il est de {}" 
      .format(round(mean_squared_error(ytrain, ytrainpred, squared=False),3), 
              round(mean_squared_error(ytest, ytestpred, squared=False),3)))

Pour les données d'entrainement le RMSE vaut 40915.894 alors que pour les données de test, il est de 38252.364


# 7. regression lineaire de ridge

In [203]:
from sklearn import linear_model
regr = linear_model.Ridge(alpha=1000)
regr.fit(xtrain, ytrain)

print("Le coefficient β1 est égal à {} tandis que l'intercept β0 est égal à {}"
      .format(round(regr.coef_[0],2),round(regr.intercept_)))

Le coefficient β1 est égal à 61.34 tandis que l'intercept β0 est égal à -57094


## 7.1 Evaluation du modèle sur le testing set

In [204]:
ytrainpred = regr.predict(xtrain)
ytestpred = regr.predict(xtest)

print ("Pour les données d'entrainement le R2 vaut {} alors que pour les données de test, il est de {}" 
      .format(round(r2_score(ytrain, ytrainpred),3), round(r2_score(ytest, ytestpred),3)))

Pour les données d'entrainement le R2 vaut 0.708 alors que pour les données de test, il est de 0.753


In [205]:
print ("Pour les données d'entrainement le RMSE vaut {} alors que pour les données de test, il est de {}" 
      .format(round(mean_squared_error(ytrain, ytrainpred, squared=False),3), 
              round(mean_squared_error(ytest, ytestpred, squared=False),3)))

Pour les données d'entrainement le RMSE vaut 43224.548 alors que pour les données de test, il est de 38172.043
